# Topological Sorting

A topological ordering of nodes in a graph is an ordering where for every pair of nodes `N,M`, if a directed edge exists from `N` to `M`, `N` precedes `M` in the ordering. Topological orderings are only possible for directed acyclic graphs (DAGs). Every DAG has at least one topological ordering. 

Suppose we had the following tree, where directed edges only exist from parent to child:
```
              1
            /   \
           /     \
          2       3
         / \     / \
        4   5   6   7
       / \   \ /     \
      8   9   10     11
```
Some valid possible topological orderings for this tree are: 
- 1, 2, 5, 4, 9, 8, 3, 7, 11, 6, 10
- 1, 3, 7, 11, 6, 2, 5, 10, 4, 9, 8

In [1]:
from typing import Union, List

class GraphNode:
    def __init__(self, value: int = None, children: Union[List, None] = None):
        self.children = children if children else []
        self.value = value


nodes = {val: GraphNode(value=val) for val in range(1,12)}
nodes[1].children += [nodes[2], nodes[3]]
nodes[2].children += [nodes[4], nodes[5]]
nodes[3].children += [nodes[6], nodes[7]]
nodes[4].children += [nodes[8], nodes[9]]
nodes[5].children += [nodes[10]]
nodes[6].children += [nodes[10]]
nodes[7].children += [nodes[11]]
root = nodes[1]


Topological orderings can be generated via Kahn's algorithm or using DFSes, and are typically `O(|V|+|E|)` for V vertices and E edges in a DAG. 

Some important considerations before implementing a topological sorting function:
- Is the graph certain to be directed and acyclic? What do we do if we detect a cycle?
- Will we be given a node/collection of nodes from which every other node is certain to be reachable? (if not, topological sorting is not possible)
- How are edges and nodes represented? How is directionality represented?
- How many children can a node have? How are those children represented (does the node have a list of children, .left and .right attributes, or something else)?
- Can we modify the input graph?

When in doubt:
- Assume nodes may have unbounded children
- Assume cycles may exist
- If performance isn't totally critical, don't modify the input graph; generate an adjacency list or collection of nodes and modify that.
- Rule out the possibility of unreachable nodes (either make sure you have a reference to every node, or that all nodes will be reachable from whatever subset you're given).


The basic DFS implementation for topological ordering requires two functions:
```
top_sort(root or list of roots):
    t_ordering = []
    visited = set()
    while list of roots is nonempty: # only need a single dfs is one root is given 
        ancestors = set()
        dfs(root, visited, ancestors, t_ordering)
    return t_ordering

dfs(node, visited, ancestors, t_ordering):
    if node in visited: return
    if node in ancestors: raise exception("cycle exists")
    ancestors.add(node)
    for each child of node:
        dfs(child, visited, ancestors, t_ordering)
    ancestors.remove(node)
    visited.add(node)
    t_ordering.prepend(node) # can use a deque for this, or just reverse the t_ordering at the end
    return 
    
```
Some common twists are:
- The graph may have cycles; return False or throw an exception if so
- The graph may have multiple roots; you need to be able to access them all to formulate the t order
- You don't get a root at the beginning but some other representation of a graph; you will need to find roots first and sometimes construct an adjacency list. 

The code below implements the DFS-based approach on the above tree. In it, we assume that the given node is the root node, and that every other node is reachable from it:

In [2]:
from collections import defaultdict
from typing import Union, List

def visit(node, visited, ancestors, top_order):
    if node in visited:
        return
    if node in ancestors:
        raise RuntimeErrror("Cycle detected, topological sorting impossible.")
    
    ancestors.add(node)
    for child in node.children:
        visit(child, visited, ancestors, top_order)
    
    ancestors.remove(node)
    visited.add(node)
    top_order.append(node.value)

        
def top_sort(root):
    top_order = []
    visit(root, set(), set(), top_order)    
    return top_order[::-1]

print(top_sort(root))


[1, 3, 7, 11, 6, 2, 5, 10, 4, 9, 8]


## [207. Course Schedule](https://leetcode.com/problems/course-schedule/)

In this question, we are given the list of edges (i.e. prereqs) and asked only to determine if topological ordering is possible (not to actually generate one); this is equivalent to asking to determine if a graph is acyclic. Since we are given a possibly acyclic graph and access to every node, we can use the DFS method to confirm every node is reachable from _some_ node and no cycles exist. By the end, we should have visited every node. Note that pairs are given as `(course, prereq)`. 

In [2]:
%run ../leetcode/207-course-schedule/solution.3.py

## [210. Course Schedule II](https://leetcode.com/problems/course-schedule-ii/)

This problem is a more typical topological sort problem; we're actually expected to produce the ordering. We can do the same operation with a minor change to produce the actual order.


In [5]:
%run ../leetcode/210-course-schedule-ii/solution.py

[0, 1]
[]
[]
[3, 0, 1, 2]


## [269. Alien Dictionary](https://leetcode.com/problems/alien-dictionary/)

#### Topological sorting is the easy part
If we can construct a DAG describing observed letter orders in the wordlist, we can use a topological sort to get  a global letter ordering (assuming the DAG is valid). The hard part for this problem is creating the DAG based on the given wordlist and finding the roots to start the tsort from. In our DAG, edge `a->b` should be added if we find `a` precedes `b` from the wordlist; we use an adjacency list to represent this. 

#### What does the wordlist tell us about the lex order?
- Word order tells us the order of their first letters (unless the letters match); seeing `["add, "bob", "car"]` implies `a < b < c`. 
- For any group of words with the same first k letters, their ordering tells us the order of their k+1 letters; seeing `["aaa, "aab", "aac"]` implies `a < b < c`.  
- To correctly execute a tsort, we only need to know the immediate predecessor and successor of a letter - in seeing `["aaa, "aab", "aac"]`, we only need to store `a < b, b < c`. We don't need to store that `a < c`.

#### How to form the DAG 
We can go word-by-word and examine the ordering of the letters. For two adjacent words `word1` and `word2`, 
if `word1[0] != word2[0]`, we can add an edge `word1[0] -> word2[0]`. If `word1[0] == word2[0]`, we must then test `word1[1] == word2[1]` and so on until we find a nonmatch or exhaust a word. If we exhaust a word, we don't store an edge - e.g. `[abc, abcd]` is the correct order but we learn nothing about `d`. Note that the wordlist is invalid if we exhausted the second word; `[abcd, abc]` is invalid. 

This technique can be used to solve [Verifying An Alien Dictionary](https://leetcode.com/problems/verifying-an-alien-dictionary/). 

#### How to get the roots
Once we have a DAG, we need to start from a node that has no parents; this is a letter that isn't found in any successor list. First, we should make sure every letter in every word is present in the DAG, even if it has no children or parents. Then, we can take the setwise complement of the keys of the adjacency list (i.e. every node in the DAG) with the union of each set of successor letters in the DAG. This leaves us with only letters who have no parents, i.e. roots. 

#### Edge cases
- Single word in wordlist. We aren't told what to do here, so I'm going to assume this is an invalid order (we have no way of knowing which letter comes first).
- Letters with no obvious order. If we get `["abc", "qmk"]`, we only know that `a` precedes `q`. I'm going to assume the remaining four letters can come in any order. 

#### A note about constraints
The constraints of the problem are small; we have at most 100 words which are each at most 100 characters (so 100,000 characters total at most), and there are only 26 possible characters. Any approach bound by the number of possible letters is `O(c)`, so we could use a quadratic algorithm on set of letters. Linear approaches looking at every given character will also easily work in under a second (quadratic ones will not, though). 



In [7]:
%run ../leetcode/269-alien-dictionary/solution.2.py